# Обучение моделей

## Импорт библиотек

In [1]:
import os, sys

PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(PROJECT_DIR)
sys.path.append(PROJECT_DIR)

/workspaces/street_tree


In [2]:
import pandas as pd 
from src.config import PATH_RAW, PATH_PROCESSED, URL, NAME_DATASET, PATH_MODELS, output_path
from src.dowload_datafraime import download_file
from src.visualization import plot_corr_matrix
from src.preprocessing import (df_fillna, split_problems, convert_to_bool, encode_and_save_categorical, 
                               load_and_encode_categorical, split_and_save)


2025-03-27 10:03:30,829 - src.utils - INFO - Logger is ready
2025-03-27 10:03:30,834 - src.utils - INFO - This is a test log message.
2025-03-27 10:03:30,835 - src.utils - INFO - PROJECT_DIR: /workspaces/street_tree
2025-03-27 10:03:30,835 - src.utils - INFO - PATH_LOGS: /workspaces/street_tree/logs
2025-03-27 10:03:30,836 - src.utils - INFO - PATH_DATA: /workspaces/street_tree/data
2025-03-27 10:03:30,837 - src.utils - INFO - PATH_RAW: /workspaces/street_tree/data/raw
2025-03-27 10:03:30,837 - src.utils - INFO - PATH_PROCESSED: /workspaces/street_tree/data/processed
2025-03-27 10:03:30,838 - src.utils - INFO - PATH_MODELS: /workspaces/street_tree/models
2025-03-27 10:03:30,839 - src.utils - INFO - PATH_REPORTS: /workspaces/street_tree/reports
2025-03-27 10:03:30,840 - src.utils - INFO - PATH_DOCS: /workspaces/street_tree/docs
2025-03-27 10:03:30,840 - src.utils - INFO - PATH_SRC: /workspaces/street_tree/src
2025-03-27 10:03:30,841 - src.utils - INFO - output_path: /workspaces/street_t

## Предобработка

In [3]:
#download_file(URL,PATH_RAW, output_path)

In [4]:
df = pd.read_csv(f'{PATH_RAW}/{NAME_DATASET}.csv')
df = df.dropna(subset=['health'])

X_train, X_test, y_train, y_test = split_and_save(df.drop('health', axis=1), df.health, output_dir=PATH_PROCESSED, size=0.1, name_train='train.csv', name_test='test.csv')

df.shape

2025-03-27 10:03:52,051 - src.utils - INFO - Data successfully saved to: /workspaces/street_tree/data/processed
2025-03-27 10:03:52,052 - src.utils - INFO - Train data shape: (586954, 44)
2025-03-27 10:03:52,053 - src.utils - INFO - Test data shape: (65218, 44)


(652172, 45)

In [5]:
X_train.columns = [col.lower().replace(' ', '_') for col in X_train.columns]
df_drop_columns = ['status', 'state', 'tree_id', 'created_at', 'stump_diam', 'address', 'spc_common',
                   'borocode', 'x_sp', 'y_sp', 'council_district', 'census_tract', 'nta_name',
                   'bin', 'zip_city', 'community_board', 'bbl']
X_train = X_train.drop(columns=df_drop_columns)
X_train.shape


(586954, 27)

In [6]:
X_train = df_fillna(X_train)
X_train.shape

/workspaces/street_tree/src/preprocessing.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['spc_latin'].fillna('No observation',inplace=True)
/workspaces/street_tree/src/preprocessing.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].metho

(586954, 27)

In [7]:
X_train = split_problems(X_train, created_columns=False)
X_train.shape

2025-03-27 10:03:55,459 - src.utils - INFO - {'metalgrates', 'sneakers', 'rootother', 'stones', 'branchlights', 'trunkother', 'noproblem', 'branchother', 'wiresrope', 'trunklights'} - уникальные проблемы


(586954, 27)

In [8]:
X_train = convert_to_bool(X_train)
X_train.shape

2025-03-27 10:03:57,771 - src.utils - INFO - Значения преобразованы в булевые


(586954, 27)

In [9]:
categorical_columns = X_train.select_dtypes(include='object').columns.tolist()
categorical_columns

['spc_latin', 'steward', 'guards', 'user_type', 'borough', 'nta']

In [10]:
X_train, label_encoders = encode_and_save_categorical(X_train, categorical_columns, PATH_MODELS, 'label_encoders.pkl')
X_train.shape

(586954, 27)

In [11]:
# Вывод значений, закодированных энкодером для столбца 'health', вместе с соответствующими им числовыми кодами
health_classes = label_encoders['spc_latin'].classes_
health_codes = label_encoders['spc_latin'].transform(health_classes)

# Создание DataFrame для удобного отображения
health_mapping = pd.DataFrame({'spc_latin': health_classes, 'Code': health_codes})
print(health_mapping)

             spc_latin  Code
0                 Acer     0
1    Acer buergerianum     1
2       Acer campestre     2
3         Acer ginnala     3
4         Acer griseum     4
..                 ...   ...
128   Tsuga canadensis   128
129    Ulmus americana   129
130   Ulmus parvifolia   130
131       Ulmus pumila   131
132    Zelkova serrata   132

[133 rows x 2 columns]


## Обучим модели

In [12]:
# Разделим X_train на обучающую и валидационную выборки и y_train на соответствующие им значения
X_train, X_val, y_train, y_val = split_and_save(X_train, y_train, output_dir=PATH_PROCESSED, size=0.2, name_train='train.csv', name_test='valid.csv')

2025-03-27 10:04:03,984 - src.utils - INFO - Data successfully saved to: /workspaces/street_tree/data/processed
2025-03-27 10:04:03,985 - src.utils - INFO - Train data shape: (469563, 27)
2025-03-27 10:04:03,987 - src.utils - INFO - Test data shape: (117391, 27)


In [13]:
y_train.unique()

array(['Poor', 'Good', 'Fair'], dtype=object)

In [14]:
import joblib

# Define the mapping
target_mapping = {'Poor': 0, 'Fair': 1, 'Good': 2}

y_train = y_train.map(target_mapping)
y_val = y_val.map(target_mapping)

# Save the mapping to a file
joblib.dump(target_mapping, f'{PATH_MODELS}/label_encoders_target.pkl')

print("Target label encoder saved to label_encoders_target.pkl")

Target label encoder saved to label_encoders_target.pkl


In [15]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
# Save the scaler
joblib.dump(scaler, f'{PATH_MODELS}/scaler.pkl')
print("Scaler saved to scaler.pkl")

Scaler saved to scaler.pkl


In [16]:
X_train.shape

(469563, 27)

In [17]:
%load_ext autoreload
%autoreload 2

from src.modeling import TabularNN

# Instantiate the model
model = TabularNN(X_train, y_train, X_val, y_val, hidden_dims=[128, 64], model_path=f'{PATH_MODELS}/tabular_model.pth')

# Train the model
model.train_model(epochs=2, learning_rate=0.001)



2025-03-27 10:04:07,139 - src.utils - INFO - Unique classes during training: [0 1 2]
2025-03-27 10:04:07,140 - src.utils - INFO - Output dimension during training: 3
2025-03-27 10:04:21,467 - src.utils - INFO - Epoch 1/2 - Train Loss: 7709.1301, Val Loss: 1896.6871, LR: 0.001000
2025-03-27 10:04:21,468 - src.utils - INFO - Accuracy: 0.5596, F1-Score: 0.6268, AUC-ROC: 0.6522
2025-03-27 10:04:21,469 - src.utils - INFO - Confusion Matrix:
2470	884	1473
6085	3975	7311
21839	14106	59248
2025-03-27 10:04:21,471 - src.utils - INFO - Best model saved based on highest AUC-ROC.
2025-03-27 10:04:34,398 - src.utils - INFO - Epoch 2/2 - Train Loss: 7614.1400, Val Loss: 1888.6606, LR: 0.001000
2025-03-27 10:04:34,399 - src.utils - INFO - Accuracy: 0.6180, F1-Score: 0.6667, AUC-ROC: 0.6552
2025-03-27 10:04:34,400 - src.utils - INFO - Confusion Matrix:
2115	974	1738
5011	3965	8395
15338	13393	66462
2025-03-27 10:04:34,402 - src.utils - INFO - Best model saved based on highest AUC-ROC.


In [18]:
import pandas as pd
import numpy as np
from src.modeling import TabularNN
from src.config import PATH_MODELS
import torch

# Define the path to the saved model
MODEL_PATH = f"{PATH_MODELS}/tabular_model.pth"

# Load the checkpoint to get the parameters
checkpoint = torch.load(MODEL_PATH)
input_dim = checkpoint['input_dim']
hidden_dims = checkpoint['hidden_dims']
output_dim = checkpoint['output_dim']
target_mapping = checkpoint['target_mapping']

# Create dummy data with the correct shape
dummy_X = pd.DataFrame(np.zeros((3, input_dim)))
dummy_y = pd.Series([0, 1, 2]) 

# Initialize the model using dummy data
loaded_model = TabularNN(dummy_X, dummy_y, dummy_X, dummy_y, hidden_dims=hidden_dims)
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model.eval()

print("Model loaded successfully!")

# Получим предсказания для dummy_X
loaded_model.predict(dummy_X)

loaded_model

/tmp/ipykernel_8482/2322857789.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_PATH)
2025-03-27 10:04:34,453 - src.utils - INFO - Unique c

Model loaded successfully!


TabularNN(
  (model): Sequential(
    (0): Linear(in_features=27, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [19]:
%load_ext autoreload
%autoreload 2
from src.modeling_catboost import CatBoostModelTrain

catboost_model = CatBoostModelTrain(model_dir=PATH_MODELS)
catboost_model.train(X_train, y_train, X_val, y_val)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'catboost'